In [ ]:
#import bybit
import time

import json
from datetime import datetime, date, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from math import floor
from termcolor import colored as cl

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (20,10)
# n=5
# m = 5
# result = []
# client = bybit.bybit()
# for i in range(n, 0, -1):
#     time = pd.Timestamp(datetime.now()-timedelta(minutes=(m*200*i)), tz='US/Pacific').timestamp()
#     res = client.LinearKline.LinearKline_get(symbol="ETHUSDT", interval=f"{m}", **{'from':time}).result()[0]['result']
#     result += res
# #print(result)


In [ ]:
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler
#-1001839931719
from dotenv import dotenv_values
import nest_asyncio
nest_asyncio.apply()

config = dotenv_values(".env")

apikey = config["TELEGRAM_API_KEY"]

async def start(update, context):
    await update.message.reply_text(f'Starting.')
    start_routine();


app = ApplicationBuilder().token(apikey).build()

app.add_handler(CommandHandler("start", start))

app.run_polling()


#next is move to a class and change self.blah to change these parameters instantly. for example alert is at -45 or 45 or whatever.
def start_routine():
    while True:
        time.sleep(1)
        current_time = pd.Timestamp(datetime.now()).second
        if current_time == 0:
            check_tsi_1m()

In [ ]:

        

#load_dotenv()  # take environment variables from .env.

In [ ]:

from datetime import datetime, timedelta
import pandas as pd

from pybit import usdt_perpetual

def get_minutes(iterations, symbol, minutes_interval):
    result = []
    session_unauth = usdt_perpetual.HTTP(
        endpoint="https://api.bybit.com"
    )
    for i in range(iterations, 0, -1):
        _time = int(pd.Timestamp(datetime.now()-timedelta(minutes=(minutes_interval*200*i)), tz='US/Pacific').timestamp())
        res = session_unauth.query_kline(
            symbol=symbol,
            interval=minutes_interval,
            from_time=_time
        )["result"]
        result += res

    df = pd.DataFrame(data=result, columns=["open_time", "high", "low","open","close","volume","turnover", "date_time"])

    df['date_time'] = df['open_time'].apply(lambda x: datetime.fromtimestamp(x))
    df.set_index('date_time').astype(float)
    return df
#print(result)

In [ ]:
eth_15m = get_minutes(15, "ETHUSDT", 15)

In [ ]:
eth_5m = get_minutes(45, "ETHUSDT", 5)

In [ ]:
eth_1m = get_minutes(135, "ETHUSDT", 1)

In [ ]:
def set_index(dataframe):
    dataframe.set_index('date_time').astype(float)

In [ ]:
calcandplot(eth_15m)

In [ ]:
calcandplot(eth_5m)

In [ ]:
#change price index so that if low[i-1]=low go with low[i] and use close price otherwise

#will want to plot this

In [ ]:
eth['close'].iloc[-100:]

In [ ]:
def setlowsandhighs(df):
    for i in range(len(df)):
        if (df['high'].iloc[i] > df['close'].iloc[i]):
            df['customclose'].iloc[i] = df['high'].iloc[i] 
        elif (df['low'].iloc[i] < df['close'].iloc[i]):
            df['customclose'].iloc[i] = df['low'].iloc[i]
        else:
            df[customclose].iloc[i] = df['close'].iloc[i]
    return df

In [ ]:
eth_15m_customclose = setlowsandhighs(eth_15m)

In [ ]:
eth_5m

In [ ]:
def mod_join(df1, df2):
    # Creating Modular Index values on df1
    n = df2.shape[0]
    df1["Modular Index"] = df1["open_time"].apply(lambda x: x)

    # Merging dataframes
    df_combined = df1.merge(df2, how="left", left_on="Modular Index", right_on="open_time")
    #df_combined
    # Dropping unnecessary columns
    df_combined = df_combined.drop(["Modular Index", "high_y", "low_y", "open_y", "turnover_y", "volume_y",'date_time_y', 'open_time_y','close_y'], axis=1)
    return df_combined
df_combined = mod_join(eth_5m, eth_15m)
df_combined[-300:]


In [ ]:
# #small merges into big
# eth_15m.set_index("date_time").astype(float)
# eth_5m.set_index("date_time").astype(float)
# #return new big merged
# def merge(small, big):
    
#     _small = small
#     _big = big
#     #_big = _big.merge(_small, left_on='date_time',right_on="date_time")
    
#     print(len(_big), len(_small))
#     _big = _big.merge(_small)
#     return _big
    
# df_merged = merge(eth_15m, eth_5m)
# df_merged

In [ ]:
# dataframe_merged

In [ ]:
def calcandplot(dataframe):
    set_index(dataframe)
    dataframe['tsi'], dataframe['signal_line'] = get_tsi(dataframe['close'], 25,13,12)
    plot(dataframe.iloc[-100:])

In [ ]:
calcTSI(eth_5m, 'tsi_5m', 'signal_line_5m')
calcTSI(eth_15m, 'tsi_15m', 'signal_line_15m')

In [ ]:
def calcTSI(df, tsi_index_name, tsi_signal_index_name ):
    df[tsi_index_name], df[tsi_signal_index_name] = get_tsi(df['close'], 25,13,12)

In [ ]:
#calculate tsi

def get_tsi(close, long, short, signal):
    diff = close - close.shift(1)
    abs_diff = abs(diff)
    
    diff_smoothed = diff.ewm(span = long, adjust = False).mean()
    diff_double_smoothed = diff_smoothed.ewm(span = short, adjust = False).mean()
    abs_diff_smoothed = abs_diff.ewm(span = long, adjust = False).mean()
    abs_diff_double_smoothed = abs_diff_smoothed.ewm(span = short, adjust = False).mean()
    
    tsi = (diff_double_smoothed / abs_diff_double_smoothed) * 100
    signal = tsi.ewm(span = signal, adjust = False).mean()
    #tsi = tsi[tsi.index >= '2020-01-01'].dropna()
    #signal = signal[signal.index >= '2020-01-01'].dropna()
    
    return tsi, signal

eth['tsi'], eth['signal_line'] = get_tsi(eth['close'], 25, 13, 12)
eth = eth[eth['date_time'] >= '2020-01-01']
eth.tail()

In [ ]:
def plot(dataframe):
    ax1 = plt.subplot2grid((11,1), (0,0), rowspan = 5, colspan = 1)
    ax2 = plt.subplot2grid((11,1), (6,0), rowspan = 5, colspan = 1)
    ax3 = plt.subplot2grid((11,1), (6,0), rowspan = 5, colspan = 1)
    ax1.plot(dataframe['close_x'], linewidth = 2)
    #ax1.plot(aapl.index, buy_price, marker = '^', markersize = 12, color = 'green', linewidth = 0, label = 'BUY SIGNAL')
    #ax1.plot(aapl.index, sell_price, marker = 'v', markersize = 12, color = 'r', linewidth = 0, label = 'SELL SIGNAL')
    #ax1.legend()
    ax1.set_title('ETH & TSI')
    ax2.plot(dataframe['tsi_5m'], linewidth = 2, color = 'orange', label = 'TSI LINE')
    ax2.plot(dataframe['signal_line_5m'], linewidth = 2, color = '#FF006E', label = 'SIGNAL LINE')
#     ax3.plot(dataframe['tsi_15m'], linewidth = 2, color = 'orange', label = 'TSI LINE')
#     ax3.plot(dataframe['signal_line_15m'], linewidth = 2, color = '#FF006E', label = 'SIGNAL LINE')
    ax2.set_title('TSI 25,13,12')
    ax2.legend()
    plt.show()

#Output:
plot(df_combined[-100:])

In [ ]:
fig = plt.figure(figsize=(5.5, 3.5), layout="constrained")
spec = fig.add_gridspec(2, 2)

ax0 = fig.add_subplot(int(df_combined['close_x']))
annotate_axes(ax0, 'ax0')

ax10 = fig.add_subplot(df_combined['tsi_5m'])
annotate_axes(ax10, 'ax10')

ax11 = fig.add_subplot(df_combined['tsi_15m'])
annotate_axes(ax11, 'ax11')

fig.suptitle('Manually added subplots, spanning a column')

In [ ]:
# //@version=2
# //Credit to https://www.tradingview.com/script/p3oqCa56-Pekipek-s-PPO-Divergence-BETA/ (I just changed the visuals and added alerts)
# study("PPO Divergence Alerts", overlay=false)

# source = open
# long_term_div = input(true, title="Use long term Divergences?")
# div_lookback_period = input(55, minval=1, title="Lookback Period")
# fastLength = input(12, minval=1), slowLength=input(26,minval=1)
# signalLength=input(9,minval=1)
# smoother = input(2,minval=1)
# fastMA = ema(source, fastLength)
# slowMA = ema(source, slowLength)
# macd = fastMA - slowMA
# macd2=(macd/slowMA)*100
# d = sma(macd2, smoother) // smoothing PPO
 
# bullishPrice = low 

# priceMins = bullishPrice > bullishPrice[1] and bullishPrice[1] < bullishPrice[2] or low[1] == low[2] and low[1] < low and low[1] < low[3] or low[1] == low[2] and low[1] == low[3] and low[1] < low and low[1] < low[4] or low[1] == low[2] and low[1] == low[3] and low[1] and low[1] == low[4] and low[1] < low and low[1] < low[5] // this line identifies bottoms and plateaus in the price
# oscMins= d > d[1] and d[1] < d[2] // this line identifies bottoms in the PPO

# BottomPointsInPPO = oscMins

# bearishPrice = high
# priceMax = bearishPrice < bearishPrice[1] and bearishPrice[1] > bearishPrice[2] or high[1] == high[2] and high[1] > high and high[1] > high[3] or high[1] == high[2] and high[1] == high[3] and high[1] > high and high[1] > high[4] or high[1] == high[2] and high[1] == high[3] and high[1] and high[1] == high[4] and high[1] > high and high[1] > high[5]  // this line identifies tops in the price
# oscMax = d < d[1] and d[1] > d[2]   // this line identifies tops in the PPO

# TopPointsInPPO = oscMax

# currenttrough4=valuewhen (oscMins, d[1], 0) // identifies the value of PPO at the most recent BOTTOM in the PPO
# lasttrough4=valuewhen (oscMins, d[1], 1) // NOT USED identifies the value of PPO at the second most recent BOTTOM in the PPO
# currenttrough5=valuewhen (oscMax, d[1], 0) // identifies the value of PPO at the most recent TOP in the PPO
# lasttrough5=valuewhen (oscMax, d[1], 1) // NOT USED identifies the value of PPO at the second most recent TOP in the PPO

# currenttrough6=valuewhen (priceMins, low[1], 0) // this line identifies the low (price) at the most recent bottom in the Price
# lasttrough6=valuewhen (priceMins, low[1], 1) // NOT USED this line identifies the low (price) at the second most recent bottom in the Price
# currenttrough7=valuewhen (priceMax, high[1], 0) // this line identifies the high (price) at the most recent top in the Price
# lasttrough7=valuewhen (priceMax, high[1], 1) // NOT USED this line identifies the high (price) at the second most recent top in the Price

# delayedlow = priceMins and barssince(oscMins) < 3 ? low[1] : na
# delayedhigh = priceMax and barssince(oscMax) < 3 ? high[1] : na

# // only take tops/bottoms in price when tops/bottoms are less than 5 bars away
# filter = barssince(priceMins) < 5 ? lowest(currenttrough6, 4) : na
# filter2 = barssince(priceMax) < 5 ? highest(currenttrough7, 4) : na

# //delayedbottom/top when oscillator bottom/top is earlier than price bottom/top
# y11 = valuewhen(oscMins, delayedlow, 0)
# y12 = valuewhen(oscMax, delayedhigh, 0)

# // only take tops/bottoms in price when tops/bottoms are less than 5 bars away, since 2nd most recent top/bottom in osc
# y2=valuewhen(oscMax, filter2, 1) // identifies the highest high in the tops of price with 5 bar lookback period SINCE the SECOND most recent top in PPO
# y6=valuewhen(oscMins, filter, 1) // identifies the lowest low in the bottoms of price with 5 bar lookback period SINCE the SECOND most recent bottom in PPO

# long_term_bull_filt = valuewhen(priceMins, lowest(div_lookback_period), 1)
# long_term_bear_filt = valuewhen(priceMax, highest(div_lookback_period), 1)

# y3=valuewhen(oscMax, currenttrough5, 0) // identifies the value of PPO in the most recent top of PPO 
# y4=valuewhen(oscMax, currenttrough5, 1) // identifies the value of PPO in the second most recent top of PPO 

# y7=valuewhen(oscMins, currenttrough4, 0) // identifies the value of PPO in the most recent bottom of PPO
# y8=valuewhen(oscMins, currenttrough4, 1) // identifies the value of PPO in the SECOND most recent bottom of PPO

# y9=valuewhen(oscMins, currenttrough6, 0)
# y10=valuewhen(oscMax, currenttrough7, 0)

# bulldiv= BottomPointsInPPO ? d[1] : na // plots dots at bottoms in the PPO
# beardiv= TopPointsInPPO ? d[1]: na // plots dots at tops in the PPO

# i = currenttrough5 < highest(d, div_lookback_period) // long term bearish oscilator divergence
# i2 = y10 > long_term_bear_filt // long term bearish top divergence
# i3 = delayedhigh > long_term_bear_filt // long term bearish delayedhigh divergence

# i4 = currenttrough4 > lowest(d, div_lookback_period) // long term bullish osc divergence
# i5 = y9 < long_term_bull_filt // long term bullish bottom div
# i6 = delayedlow < long_term_bull_filt // long term bullish delayedbottom div


# plot(0, color=gray)
# plot(d, color=black)
# plot(bulldiv, title = "Bottoms", color=maroon, style=circles, linewidth=3, offset= -1)
# plot(beardiv, title = "Tops", color=green, style=circles, linewidth=3, offset= -1)

# bearishdiv1 = (y10 > y2 and oscMax and y3 < y4) ? true : false
# bearishdiv2 = (delayedhigh > y2 and y3 < y4) ? true : false
# bearishdiv3 = (long_term_div and oscMax and i and i2) ? true : false
# bearishdiv4 = (long_term_div and i and i3) ? true : false

# bullishdiv1 = (y9 < y6 and oscMins and y7 > y8) ? true : false
# bullishdiv2 = (delayedlow < y6 and y7 > y8) ? true : false
# bullishdiv3 = (long_term_div and oscMins and i4 and i5) ? true : false
# bullishdiv4 = (long_term_div and i4 and i6) ? true : false

# bearish = bearishdiv1 or bearishdiv2 or bearishdiv3 or bearishdiv4
# bullish = bullishdiv1 or bullishdiv2 or bullishdiv3 or bullishdiv4
 
# greendot = beardiv != 0 ? true : false
# reddot = bulldiv != 0 ? true : false

# alertcondition( bearish, title="Bearish Signal (Orange Circle)", message="Orange & Bearish: Short " ) 
# alertcondition( bullish, title="Bullish Signal (Purple Circle)", message="Purple & Bullish: Long " )
# alertcondition( greendot, title="PPO High (Green Dot)", message="Green High Point: Short " ) 
# alertcondition( reddot, title="PPO Low (Red Dot)", message="Red Low Point: Long " )

# plot(y10>y2 and oscMax and y3 < y4 ? d :na, title = "Bearish Divergence1", color=orange, style= circles, linewidth=6)
# plot(y9<y6 and oscMins and y7 > y8 ? d :na, title = "Bullish Divergence1", color=purple, style=circles, linewidth=6)
# plot(delayedhigh>y2 and y3 < y4 ? d :na, title = "Bearish Divergence2", color=orange, style= circles, linewidth=6)
# plot(delayedlow<y6 and y7 > y8 ? d :na, title = "Bullish Divergence2", color=purple, style=circles, linewidth=6)

# plot(long_term_div and oscMax and i and i2 ? d :na, title = "Bearish Divergence3", color=orange, style= circles, linewidth=6)
# plot(long_term_div and oscMins and i4 and i5 ? d : na, title = "Bullish Divergence3", color=purple, style=circles, linewidth=6)
# plot(long_term_div and i and i3 ? d :na, title = "Bearish Divergence4", color=orange, style= circles, linewidth=6)
# plot(long_term_div and i4 and i6 ? d : na, title = "Bullish Divergence4", color=purple, style=circles, linewidth=6)